# Linear Regression

## Imports

Remember to start by running

```shell
uv sync
````

in the directory of the repo after cloning.

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# Load the necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from TA1_linreg import LinearRegressionModel
import time

In [39]:
# Set the random seed for reproducibility
# np.random.seed() has a range of [0, 2**32 - 1] for the seed value
# print(np.random.randint(0, 2**32 - 1)) # Uncomment here for the first run to be a statistical purist
SEED = 1687184344 # Obviously your print is going to be something different

np.random.seed(SEED)
torch.manual_seed(SEED)

start_time = time.time()

In [40]:
# Load the data (House Prices dataset from Kaggle)

df = pd.read_csv("../data/house_prices/house_prices_train.csv")

In [41]:
# Take a peek at the data (Target Variable is SalePrice, self-descriptive)

df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## One Neuron Model

In [42]:
"""
We create a One Punch Model, handling standardization as well as the training loop.

WARNING: This is such a simple model that I am doing the Pytorch loop so it resembles Sklearn's fit method.
Just ignore my class structure here, I'm going for practicality over scalability of the structure.
To be clear, you should NOT build your neural networks like this when using more than a single neuron.
"""

class OnePunchModel:
    def __init__(self, input_dim=1,learning_rate=0.01, epochs=100, standardize=True, verbose=True):
        self.input_dim = input_dim
        self.lr = learning_rate
        self.epochs = epochs
        self.std = standardize
        self.verbose = verbose
        
        self.model = nn.Linear(in_features=self.input_dim, out_features=1)
        self.loss = nn.MSELoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr)
    
    def fit(self, X, y):
        if self.std:
            self.X_mean, self.X_std = X.mean(dim=0), X.std(dim=0)
            self.y_mean, self.y_std = y.mean(), y.std()
            X = (X - self.X_mean) / self.X_std
            y = (y - self.y_mean) / self.y_std
        
        for epoch in range(self.epochs):
            self.model.train()
            
            outputs = self.model(X)
            
            mse = self.loss(outputs, y)
            
            self.optimizer.zero_grad()
            mse.backward()
            self.optimizer.step()
            
            if self.verbose and (epoch-1) % 1000 == 0:
                print(f"Epoch [{epoch+1}/{self.epochs}], Loss(MSE): {mse.item():.4f}")
        
        return self
    
    def get_weights(self):
        weights = self.model.weight.detach().numpy()
        bias = self.model.bias.detach().numpy()
        
        # Before continuing, we need to de-standardize the weights and bias if standardization was applied during training
        if self.std:
            xbar, ybar, sx, sy = self.X_mean.detach().numpy(), self.y_mean.item(), self.X_std.detach().numpy(), self.y_std.item()
            weights = weights * sy / sx
            bias = sy * bias + ybar - (weights * xbar).sum()
        
        # We create a dictionary where the bias is stored as the first key (beta_0) and the weights go from beta_1 to beta_n
        betas_dict = {"beta_0": bias[0].item()}
        for i, w in enumerate(weights[0]):
            betas_dict[f"beta_{i+1}"] = w.item()
        
        return betas_dict


In [43]:
"""
Function for the comparison tables, not too relevant to understand it.
"""

def compare_parameters(dict1, name1, dict2, name2, dict3, name3, dict4, name4):
    """
    Creates a styled comparison table for four sets of model parameters.
    Args:
        dict1, dict2, dict3, dict4: Dictionaries with matching keys (parameter names).
        name1, name2, name3, name4: Labels for each set of parameters.
    """
    # 1. Create a list of the dictionaries and their corresponding names as indices
    data = [dict1, dict2, dict3, dict4]
    indices = [name1, name2, name3, name4]
    # 2. Convert to a DataFrame (Keys become columns, Names become rows)
    comparison_df = pd.DataFrame(data, index=indices)
    
    # 3. Apply styling
    # - background_gradient: Highlights differences in values across columns
    # - format: Rounds to 2 decimal places for readability
    # - set_table_styles: Adds padding and better font sizing for the UI
    styled_comparison = comparison_df.style \
        .background_gradient(cmap='RdYlGn', axis=0) \
        .format("{:,.2f}") \
        .set_caption("Comparison of Model Weights and Biases") \
        .set_table_styles([
            {'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center'), ('background-color', "#10365c")]},
            {'selector': 'td', 'props': [('text-align', 'center'), ('padding', '12px'), ('font-family', 'monospace')]},
            {'selector': 'caption', 'props': [('font-size', '16pt'), ('font-weight', 'bold'), ('padding', '10px')]}
        ])
    
    return styled_comparison

In [44]:
# TRAIN TEST SPLIT

# Shuffle the data so we don't have ordering artifacts
df_shuffled = df.sample(frac=1, random_state=168).reset_index(drop=True)
# display(df)
# display(df_shuffled)

# Split the data until the 80% mark (80 train, 20 test)
print(len(df)*0.8) # 1168

df_train, df_test = df_shuffled.iloc[:1168, :].copy(), df_shuffled.iloc[1168:, :].copy()

print(len(df_train), len(df_test)) # 1168 292

# display(df_train.head(), df_test.head())

1168.0
1168 292


## Univariate Linear Regression

In [45]:
# We are going to start by focusing only on 1 variable, GrLivArea (Squared Feet)

X_train, X_test = df_train[["GrLivArea", ]].copy(), df_test[["GrLivArea"]].copy()
y_train, y_test = df_train["SalePrice"].copy(), df_test["SalePrice"].copy()

X_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

print(X_train.shape, X_test.shape) # (1168, 1) (292, 1)
print(X_tensor.shape, y_tensor.shape) # torch.Size([1168, 1]) torch.Size([1168, 1])

(1168, 1) (292, 1)
torch.Size([1168, 1]) torch.Size([1168, 1])


In [46]:
linreg = LinearRegressionModel(
                learning_rate = 0.05,
                n_iterations = 10000,
                tol = 1e-6,
                verbose = False,
                standardize = True
            )

linreg.fit(X_train, y_train)

linreg.fit_closed_form_linreg(X_train, y_train)

linreg.fit_svd_linreg(X_train, y_train)

fit_params = linreg.get_params()
display(fit_params)

{'params_gd': {'beta_0': Array(18044.15768823, dtype=float64),
  'beta_1': Array(108.03859363, dtype=float64)},
 'converged': 'Converged',
 'n_iter': 136,
 'final_loss': 0.499224242894264,
 'closed_form_params': {'beta_0': Array(18044.05985916, dtype=float64),
  'beta_1': Array(108.03865828, dtype=float64)},
 'svd_params': {'beta_0': Array(18044.05985916, dtype=float64),
  'beta_1': Array(108.03865828, dtype=float64)}}

In [47]:
neuralnet = OnePunchModel(
                input_dim = X_tensor.shape[1],
                learning_rate = 0.05,
                epochs = 10000,
                standardize = True,
                verbose = True
            )

neuralnet.fit(X_tensor, y_tensor)
nn_params = neuralnet.get_weights()
print(nn_params)

Epoch [2/10000], Loss(MSE): 1.4850
Epoch [1002/10000], Loss(MSE): 0.4988
Epoch [2002/10000], Loss(MSE): 0.4988
Epoch [3002/10000], Loss(MSE): 0.4988
Epoch [4002/10000], Loss(MSE): 0.4988
Epoch [5002/10000], Loss(MSE): 0.4988
Epoch [6002/10000], Loss(MSE): 0.4988
Epoch [7002/10000], Loss(MSE): 0.4988
Epoch [8002/10000], Loss(MSE): 0.4988
Epoch [9002/10000], Loss(MSE): 0.4988
{'beta_0': 18044.125, 'beta_1': 108.03861999511719}


In [48]:
comparison = compare_parameters(
    dict1=fit_params['params_gd'], name1="Gradient Descent",
    dict2=fit_params['closed_form_params'], name2="Closed-Form",
    dict3=fit_params['svd_params'], name3="SVD",
    dict4=nn_params, name4="One Neuron NN"
)
display(comparison)

,beta_0,beta_1
Gradient Descent,"18,044.16",108.04
Closed-Form,"18,044.06",108.04
SVD,"18,044.06",108.04
One Neuron NN,"18,044.12",108.04


## Multivariate Linear Regression

In [49]:
# And now we are going to try the multivariate case using both: 
# GrLivArea (Squared Feet) and OverallQual (Overall Quality)

# X_train, X_test = df_train[["GrLivArea", "OverallQual"]].copy(), df_test[["GrLivArea", "OverallQual"]].copy()
X_train, X_test = df_train[["GrLivArea", "OverallQual", "GarageCars", "FullBath"]].copy(), df_test[["GrLivArea", "OverallQual", "GarageCars", "FullBath"]].copy()
y_train, y_test = df_train["SalePrice"].copy(), df_test["SalePrice"].copy()

X_tensor  = torch.tensor(X_train.values, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

print(X_train.shape, X_test.shape)
print(X_tensor.shape, y_tensor.shape)

(1168, 4) (292, 4)
torch.Size([1168, 4]) torch.Size([1168, 1])


In [50]:
linreg = LinearRegressionModel(
                learning_rate = 0.05,
                n_iterations = 10000,
                tol = 1e-6,
                verbose = False,
                standardize = True
            )

linreg.fit(X_train, y_train)

linreg.fit_closed_form_linreg(X_train, y_train)

linreg.fit_svd_linreg(X_train, y_train)

fit_params = linreg.get_params()
display(fit_params)

{'params_gd': {'beta_0': Array(-98936.05047602, dtype=float64),
  'beta_1': Array(52.69664682, dtype=float64),
  'beta_2': Array(27097.21745759, dtype=float64),
  'beta_3': Array(21349.86490647, dtype=float64),
  'beta_4': Array(-1702.22081411, dtype=float64)},
 'converged': 'Converged',
 'n_iter': 321,
 'final_loss': 0.2681483644438423,
 'closed_form_params': {'beta_0': Array(-98936.19499883, dtype=float64),
  'beta_1': Array(52.69663136, dtype=float64),
  'beta_2': Array(27097.28031199, dtype=float64),
  'beta_3': Array(21349.79361888, dtype=float64),
  'beta_4': Array(-1702.27787876, dtype=float64)},
 'svd_params': {'beta_0': Array(-98936.19394482, dtype=float64),
  'beta_1': Array(52.69663133, dtype=float64),
  'beta_2': Array(27097.2801511, dtype=float64),
  'beta_3': Array(21349.79366824, dtype=float64),
  'beta_4': Array(-1702.27792536, dtype=float64)}}

In [51]:
neuralnet = OnePunchModel(
                input_dim = X_tensor.shape[1],
                learning_rate = 0.05,
                epochs = 10000,
                standardize = True,
                verbose = True
            )

neuralnet.fit(X_tensor, y_tensor)
nn_params = neuralnet.get_weights()
print(nn_params)

Epoch [2/10000], Loss(MSE): 0.9581
Epoch [1002/10000], Loss(MSE): 0.2679
Epoch [2002/10000], Loss(MSE): 0.2679
Epoch [3002/10000], Loss(MSE): 0.2679
Epoch [4002/10000], Loss(MSE): 0.2679
Epoch [5002/10000], Loss(MSE): 0.2679
Epoch [6002/10000], Loss(MSE): 0.2679
Epoch [7002/10000], Loss(MSE): 0.2679
Epoch [8002/10000], Loss(MSE): 0.2679
Epoch [9002/10000], Loss(MSE): 0.2679
{'beta_0': -98936.234375, 'beta_1': 52.69657897949219, 'beta_2': 27097.302734375, 'beta_3': 21349.775390625, 'beta_4': -1702.2650146484375}


In [52]:
comparison = compare_parameters(
    dict1=fit_params['params_gd'], name1="Gradient Descent",
    dict2=fit_params['closed_form_params'], name2="Closed-Form",
    dict3=fit_params['svd_params'], name3="SVD",
    dict4=nn_params, name4="One Neuron NN"
)
display(comparison)

,beta_0,beta_1,beta_2,beta_3,beta_4
Gradient Descent,"-98,936.05",52.70,"27,097.22","21,349.86","-1,702.22"
Closed-Form,"-98,936.19",52.70,"27,097.28","21,349.79","-1,702.28"
SVD,"-98,936.19",52.70,"27,097.28","21,349.79","-1,702.28"
One Neuron NN,"-98,936.23",52.70,"27,097.30","21,349.78","-1,702.27"


In [53]:
end_time = time.time()

print(f"Total execution time: {end_time - start_time:.4f} seconds")

Total execution time: 2.6361 seconds
